<a href="https://colab.research.google.com/github/sitaarn/Big-Data/blob/main/Big%20Data%20Tugas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tugas 1

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min

# Inisialisasi Spark Session
spark = SparkSession.builder.appName('TugasPertemuan3').getOrCreate()

data = [
    ('Rini', 'IT', 8000000),
    ('Dika', 'Sales', 4500000),
    ('Bagus', 'IT', 5200000),
    ('Andini', 'Finance', 3000000),
    ('Raka', 'Finance', 4800000),
    ('Sabila', 'Marketing', 3500000),
    ('Janice', 'Marketing', 3700000)
]

columns = ['Name', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)

# Tampilkan DataFrame
print("Data Awal:")
df.show()

# Menampilkan schema
print("Schema DataFrame:")
df.printSchema()


Data Awal:
+------+----------+-------+
|  Name|Department| Salary|
+------+----------+-------+
|  Rini|        IT|8000000|
|  Dika|     Sales|4500000|
| Bagus|        IT|5200000|
|Andini|   Finance|3000000|
|  Raka|   Finance|4800000|
|Sabila| Marketing|3500000|
|Janice| Marketing|3700000|
+------+----------+-------+

Schema DataFrame:
root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)



In [ ]:
# Tugas 2

# Select kolom tertentu
print("Data Nama dan Gaji:")
df.select("Name", "Salary").show()

# Filter data dengan kondisi
print("Karyawan dengan Gaji > 3500000:")
df.filter(col("Salary") > 3500000).show()

# GroupBy + Agregasi
print("Rata-rata Gaji Tiap Departemen:")
df.groupBy("Department").agg(avg("Salary").alias("MeanSalary")).show()

print("Gaji Max Tiap Departemen:")
df.groupBy("Department").agg(max("Salary").alias("MaxSalary")).show()

print("Gaji Min Tiap Departemen:")
df.groupBy("Department").agg(min("Salary").alias("MinSalary")).show()

print("Total gaji per departemen:")
df.groupBy("Department").agg(sum("Salary").alias("TotalSalary")).show()

Data Nama dan Gaji:
+------+-------+
|  Name| Salary|
+------+-------+
|  Rini|8000000|
|  Dika|4500000|
| Bagus|5200000|
|Andini|3000000|
|  Raka|4800000|
|Sabila|3500000|
|Janice|3700000|
+------+-------+

Karyawan dengan Gaji > 3500000:
+------+----------+-------+
|  Name|Department| Salary|
+------+----------+-------+
|  Rini|        IT|8000000|
|  Dika|     Sales|4500000|
| Bagus|        IT|5200000|
|  Raka|   Finance|4800000|
|Janice| Marketing|3700000|
+------+----------+-------+

Rata-rata Gaji Tiap Departemen:
+----------+----------+
|Department|MeanSalary|
+----------+----------+
|     Sales| 4500000.0|
|        IT| 6600000.0|
|   Finance| 3900000.0|
| Marketing| 3600000.0|
+----------+----------+

Gaji Max Tiap Departemen:
+----------+---------+
|Department|MaxSalary|
+----------+---------+
|     Sales|  4500000|
|        IT|  8000000|
|   Finance|  4800000|
| Marketing|  3700000|
+----------+---------+

Gaji Min Tiap Departemen:
+----------+---------+
|Department|MinSalary|

In [ ]:
# Tugas3

from pyspark.sql.functions import col, struct, array, explode, lit

# --- Manipulasi sederhana (numeric) ---
print("Tambah kolom SalaryBonus (10% dari Salary):")
df = df.withColumn('SalaryBonus', col('Salary') * 0.1)
df.show()

print("Tambah kolom TotalCompensation (Salary + Bonus):")
df = df.withColumn('TotalCompensation', col('Salary') + col('SalaryBonus'))
df.show()

# --- Manipulasi tipe data kompleks ---
# 1. Struct: gabungkan beberapa kolom jadi satu
print("Gabungkan kolom menjadi Struct:")
df_struct = df.withColumn("EmployeeInfo", struct("Name", "Department"))
df_struct.select("EmployeeInfo", "Salary").show(truncate=False)

# 2. Array: simpan beberapa nilai dalam satu kolom array
print("Buat kolom array:")
df_array = df.withColumn("SalaryComponents", array("Salary", "SalaryBonus", "TotalCompensation"))
df_array.select("Name", "SalaryComponents").show(truncate=False)

# 3. Explode: pecah array menjadi baris terpisah
print("Pecah kolom array menjadi baris terpisah:")
df_exploded = df_array.select("Name", explode("SalaryComponents").alias("ComponentValue"))
df_exploded.show()



Tambah kolom SalaryBonus (10% dari Salary):
+------+----------+-------+-----------+-----------------+
|  Name|Department| Salary|SalaryBonus|TotalCompensation|
+------+----------+-------+-----------+-----------------+
|  Rini|        IT|8000000|   800000.0|        8800000.0|
|  Dika|     Sales|4500000|   450000.0|        4950000.0|
| Bagus|        IT|5200000|   520000.0|        5720000.0|
|Andini|   Finance|3000000|   300000.0|        3300000.0|
|  Raka|   Finance|4800000|   480000.0|        5280000.0|
|Sabila| Marketing|3500000|   350000.0|        3850000.0|
|Janice| Marketing|3700000|   370000.0|        4070000.0|
+------+----------+-------+-----------+-----------------+

Tambah kolom TotalCompensation (Salary + Bonus):
+------+----------+-------+-----------+-----------------+
|  Name|Department| Salary|SalaryBonus|TotalCompensation|
+------+----------+-------+-----------+-----------------+
|  Rini|        IT|8000000|   800000.0|        8800000.0|
|  Dika|     Sales|4500000|   450000

In [ ]:
# Tugas 4

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# WindowSpec: dipartisi berdasarkan Department, diurutkan berdasarkan Salary
windowSpec = Window.partitionBy("Department").orderBy("Salary")

# --- Ranking (Rank) ---
print("Ranking gaji per Department:")
df_rank = df.withColumn("Rank", F.rank().over(windowSpec))
df_rank.show()

# --- Dense Rank ---
print("Dense Rank (tidak lompat nomor kalau ada gaji sama):")
df_dense = df.withColumn("DenseRank", F.dense_rank().over(windowSpec))
df_dense.show()

# --- Row Number ---
print("Row Number per Department:")
df_rownum = df.withColumn("RowNumber", F.row_number().over(windowSpec))
df_rownum.show()

# --- Running Total (Cumulative Sum) ---
print("Running Total Salary per Department:")
df_running = df.withColumn("RunningTotal", F.sum("Salary").over(windowSpec))
df_running.show()

# --- Moving Average (contoh tambahan) ---
print("Moving Average Salary per Department:")
df_mavg = df.withColumn("MovingAvg", F.avg("Salary").over(windowSpec))
df_mavg.show()


Ranking gaji per Department:
+------+----------+-------+-----------+-----------------+----+
|  Name|Department| Salary|SalaryBonus|TotalCompensation|Rank|
+------+----------+-------+-----------+-----------------+----+
|Andini|   Finance|3000000|   300000.0|        3300000.0|   1|
|  Raka|   Finance|4800000|   480000.0|        5280000.0|   2|
| Bagus|        IT|5200000|   520000.0|        5720000.0|   1|
|  Rini|        IT|8000000|   800000.0|        8800000.0|   2|
|Sabila| Marketing|3500000|   350000.0|        3850000.0|   1|
|Janice| Marketing|3700000|   370000.0|        4070000.0|   2|
|  Dika|     Sales|4500000|   450000.0|        4950000.0|   1|
+------+----------+-------+-----------+-----------------+----+

Dense Rank (tidak lompat nomor kalau ada gaji sama):
+------+----------+-------+-----------+-----------------+---------+
|  Name|Department| Salary|SalaryBonus|TotalCompensation|DenseRank|
+------+----------+-------+-----------+-----------------+---------+
|Andini|   Finance|3

In [1]:
# Tugas 5

import kagglehub

# Download latest version
path = kagglehub.dataset_download("msnbehdani/mock-dataset-of-second-hand-car-sales")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mock-dataset-of-second-hand-car-sales' dataset.
Path to dataset files: /kaggle/input/mock-dataset-of-second-hand-car-sales


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

# Inisialisasi Spark
spark = SparkSession.builder.appName("TugasPertemuan3_5").getOrCreate()

# Path dataset hasil download dari kagglehub
path = "/kaggle/input/mock-dataset-of-second-hand-car-sales"  # ganti sesuai output kagglehub
file_path = os.path.join(path, "car_sales_data.csv")  # cek nama file csv di folder

# Load CSV ke DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Tampilkan data awal
print("Data sample:")
df.show(7)
df.printSchema()

# -----------------------------
# Contoh manipulasi data
# -----------------------------

# Jumlah total baris (mobil yang dijual)
df.select(F.count("*").alias("TotalRecords")).show()

# Distribusi mobil berdasarkan Manufacturer
df.groupBy("Manufacturer").count().orderBy(F.desc("count")).show(10)

# Harga rata-rata per Manufacturer
df.groupBy("Manufacturer").agg(F.avg("Price").alias("AveragePrice")).orderBy(F.desc("AveragePrice")).show(10)

# Tahun produksi mobil tertua & termuda
df.select(F.min("Year of manufacture").alias("OldestCar"),
          F.max("Year of manufacture").alias("NewestCar")).show()

# Rata-rata Mileage per Manufacturer
df.groupBy("Manufacturer").agg(F.avg("Mileage").alias("AvgMileage")).orderBy("AvgMileage").show(10)

# Total penjualan per tahun
df.groupBy("Year of manufacture").agg(F.count("*").alias("TotalSales")).orderBy("Year of manufacture").show(10)


Data sample:
+------------+----------+-----------+---------+-------------------+-------+-----+
|Manufacturer|     Model|Engine size|Fuel type|Year of manufacture|Mileage|Price|
+------------+----------+-----------+---------+-------------------+-------+-----+
|        Ford|    Fiesta|        1.0|   Petrol|               2002| 127300| 3074|
|     Porsche|718 Cayman|        4.0|   Petrol|               2016|  57850|49704|
|        Ford|    Mondeo|        1.6|   Diesel|               2014|  39190|24072|
|      Toyota|      RAV4|        1.8|   Hybrid|               1988| 210814| 1705|
|          VW|      Polo|        1.0|   Petrol|               2006| 127869| 4101|
|        Ford|     Focus|        1.4|   Petrol|               2018|  33603|29204|
|        Ford|    Mondeo|        1.8|   Diesel|               2010|  86686|14350|
+------------+----------+-----------+---------+-------------------+-------+-----+
only showing top 7 rows

root
 |-- Manufacturer: string (nullable = true)
 |-- Model: